In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from keras import Sequential
from keras.models import Model
from keras.layers import LSTM, Dense, Concatenate, Flatten, Input
from keras.layers.merge import concatenate
from keras.optimizers import Adam
from keras.losses import logcosh
import tensorflow as tf
from keras.utils import plot_model
%matplotlib inline
np.random.seed(2018)

Using TensorFlow backend.


In [2]:
#Population values
population = {
    "Chiapas_2016-2017.csv": 5217908,
    "Colima_2016-2017.csv": 711235,
    "Guerrero_2016-2017.csv": 3533251,
    "Hidalgo_2016-2017.csv": 2858359,
    "NuevoLeon_2016-2017.csv": 5119504,
    "Oaxaca_2016-2017.csv": 3967889,
    "QuintanaRoo_2016-2017.csv": 1501562,
    "Tabasco_2016-2017.csv" : 2395272,
    "Veracruz_2016-2017.csv" : 8112505,
    "Yucatan_2016-2017.csv" : 2097175,
    
    "casanare_2016-2017.csv" : 356438,
    "cordoba_2016-2017.csv" : 1709603,
    "cundinamarca_2016-2017.csv" : 2680041,
    "huila_2016-2017.csv" : 1154804,
    "meta_2016-2017.csv" : 961292,
    "santander_2016-2017.csv" : 2061095,
    "santander_norte_2016-2017.csv" : 1355723,
    "tolima_2016-2017.csv" : 1408274,
    "valle_cauca_2016-2017.csv" : 4613377,
    
    "Alagoas_2016-2017.csv": 3375823,
    "Bahia_2016-2017.csv": 15344447,
    "Ceara_2016-2017.csv": 9020460,
    "Goias_2016-2017.csv": 6778772,
    "Maranhao_2016-2017.csv": 7000229,
    "MatoGrosso_2016-2017.csv": 3344544,
    "MinasGerais_2016-2017.csv": 21119536,
    "Para_2016-2017.csv": 8366628,
    "RioDeJaneiro_2016-2017.csv": 16718956,
    "SaoPaulo_2016-2017.csv": 45094866,
}

In [3]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [4]:
def getXY(dataset, scale):
    dataset[["Searches"]] /= 100
    dataset[["Cases"]] = dataset[["Cases"]].apply(lambda x: x*100000/scale, axis=1)
    
    dataset.drop(["Date"], axis=1, inplace=True)
    values = dataset.values.astype("float32")
    
    

    n_weeks = 4
    n_features = 2

    reframed = series_to_supervised(values, n_weeks, 1)
    values = reframed.values
    print("Reframed Shape: ", reframed.shape)
    totalFeatures = reframed.shape[1]
    n_obs = n_weeks * n_features

    x, y = values[:, :-2], values[:, -1] # Pick last week's cases as y and drop last week's 

    x = x.reshape((x.shape[0], n_weeks, n_features)) # Reshape as 3-D
    return x, y 

In [5]:
def saveModel(model, modelName):
    jsonName = "{}.json".format(modelName)
    h5Name = "{}.h5".format(modelName)

    model_json = model.to_json()
    with open(jsonName, "w") as json_file:
        json_file.write(model_json)
    #seralize weights to HDF5
    model.save_weights(h5Name)

In [6]:
def createModel(modelName):
    jsonName = "{}.json".format(modelName)
    h5Name = "{}.h5".format(modelName)
    
    
    input_layer = Input(shape=(4,2))
    b1_out = LSTM(512, return_sequences=True)(input_layer)
#     b1_out = LSTM(512, activation="tanh", return_sequences=True)(b1_out)
    b1_out = LSTM(256, activation="tanh", return_sequences=False)(b1_out)
    
    model1 = Model(input_layer, b1_out)
    
    b2_out = Dense(1024, activation="tanh")(input_layer)
#     b2_out = Dense(256, activation="tanh")(b2_out)
    b2_out = Flatten()(b2_out)
    
    model2 = Model(input_layer, b2_out)
    
    concatenated = concatenate([b1_out, b2_out])
    out = Dense(128, activation="tanh")(concatenated)
    out = Dense(1, activation='linear', name='output_layer')(concatenated)
    
    model = Model([input_layer], out)
    model.compile(loss=["mae"], optimizer="adam", metrics=["mae"])
    
    return model

In [7]:
for country in ["Mexico", "Brazil", "Colombia"]:
    folder = "../../data/{}/processed_data".format(country)
    files = os.listdir(folder)
    for file in files:
        
        with tf.Session() as sess:
            dataset = pd.read_csv("{}/{}".format(folder, file))
            state = file

            if(not os.path.isdir("{}/{}".format(country, file))):
                os.mkdir("{}/{}".format(country, file))

            train = dataset[:52]
            test = dataset[48:] # Keep 4 previous values to be able to predict all 52 weeks of next year
            x, y = getXY(train, population[file])
            test_x, test_y = getXY(test, population[file])
            model = createModel(file)
            model.fit(x, y,
                    epochs = 100,
                    batch_size=x.shape[0],
                    verbose=1, 
                    shuffle=True,
                    validation_data=(test_x,test_y)
                     )

            
            predictions = model.predict(test_x)
            #Transform to 1-D
            test_y = test_y.reshape((len(test_y), 1))

            #Rescale
            inv_yPred = np.apply_along_axis(lambda x: x * population[file] / 100000, 1, predictions)
            inv_y = np.apply_along_axis(lambda x: x * population[file] / 100000, 1, test_y)

            test = test[4:] # drop the first 4 values used previously to be able to predict the full year

            test["Cases"] = inv_y
            test["predictions"] = inv_yPred
            test["error"] = test["Cases"] - test["predictions"]
            test.to_csv("{}/{}/Predictions.csv".format(country, file))
            test[["Cases", "predictions"]].plot(figsize=(10,10))
            plt.title(file)
            plt.xlabel("Week")
            plt.ylabel("Cases")
            plt.legend()
            plt.grid()
            fig = plt.gcf()
            fig.savefig("{}/{}/Obs-Pred.png".format(country, file))
            plt.grid()
            plt.close("all")

            saveModel(model, "{}/{}/Model".format(country, file))

E:\Aplicaciones\Anaconda\lib\site-packages\pandas\core\frame.py:2352: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
E:\Aplicaciones\Anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


Reframed Shape:  (48, 10)
Reframed Shape:  (52, 10)
Train on 48 samples, validate on 52 samples
Epoch 1/100
48/48 [==============================] - 2s 43ms/step - loss: 0.2796 - mean_absolute_error: 0.2796 - val_loss: 0.0525 - val_mean_absolute_error: 0.0525
Epoch 2/100
48/48 [==============================] - 0s 1ms/step - loss: 0.2340 - mean_absolute_error: 0.2340 - val_loss: 0.0912 - val_mean_absolute_error: 0.0912
Epoch 3/100
48/48 [==============================] - 0s 958us/step - loss: 0.2645 - mean_absolute_error: 0.2645 - val_loss: 0.0863 - val_mean_absolute_error: 0.0863
Epoch 4/100
48/48 [==============================] - 0s 979us/step - loss: 0.2658 - mean_absolute_error: 0.2658 - val_loss: 0.0619 - val_mean_absolute_error: 0.0619
Epoch 5/100
48/48 [==============================] - 0s 958us/step - loss: 0.2432 - mean_absolute_error: 0.2432 - val_loss: 0.0306 - val_mean_absolute_error: 0.0306
Epoch 6/100
48/48 [==============================] - 0s 1ms/step - loss: 0.2195 - 

48/48 [==============================] - 0s 979us/step - loss: 0.2074 - mean_absolute_error: 0.2074 - val_loss: 0.0273 - val_mean_absolute_error: 0.0273
Epoch 51/100
48/48 [==============================] - 0s 958us/step - loss: 0.2072 - mean_absolute_error: 0.2072 - val_loss: 0.0272 - val_mean_absolute_error: 0.0272
Epoch 52/100
48/48 [==============================] - 0s 979us/step - loss: 0.2071 - mean_absolute_error: 0.2071 - val_loss: 0.0271 - val_mean_absolute_error: 0.0271
Epoch 53/100
48/48 [==============================] - 0s 917us/step - loss: 0.2070 - mean_absolute_error: 0.2070 - val_loss: 0.0265 - val_mean_absolute_error: 0.0265
Epoch 54/100
48/48 [==============================] - 0s 958us/step - loss: 0.2072 - mean_absolute_error: 0.2072 - val_loss: 0.0256 - val_mean_absolute_error: 0.0256
Epoch 55/100
48/48 [==============================] - 0s 958us/step - loss: 0.2075 - mean_absolute_error: 0.2075 - val_loss: 0.0251 - val_mean_absolute_error: 0.0251
Epoch 56/100
48/4

Epoch 100/100
48/48 [==============================] - 0s 1000us/step - loss: 0.2068 - mean_absolute_error: 0.2068 - val_loss: 0.0262 - val_mean_absolute_error: 0.0262


E:\Aplicaciones\Anaconda\lib\site-packages\pandas\core\frame.py:2352: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
E:\Aplicaciones\Anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


...

Epoch 84/100
48/48 [==============================] - 0s 1ms/step - loss: 2.6082 - mean_absolute_error: 2.6082 - val_loss: 0.5883 - val_mean_absolute_error: 0.5883
Epoch 85/100
48/48 [==============================] - 0s 1ms/step - loss: 2.4971 - mean_absolute_error: 2.4971 - val_loss: 0.6335 - val_mean_absolute_error: 0.6335
Epoch 86/100
48/48 [==============================] - 0s 1ms/step - loss: 2.4846 - mean_absolute_error: 2.4846 - val_loss: 0.6426 - val_mean_absolute_error: 0.6426
Epoch 87/100
48/48 [==============================] - 0s 1ms/step - loss: 2.4473 - mean_absolute_error: 2.4473 - val_loss: 0.6518 - val_mean_absolute_error: 0.6518
Epoch 88/100
48/48 [==============================] - 0s 1ms/step - loss: 2.3941 - mean_absolute_error: 2.3941 - val_loss: 0.6859 - val_mean_absolute_error: 0.6859
Epoch 89/100
48/48 [==============================] - 0s 1ms/step - loss: 2.3142 - mean_absolute_error: 2.3142 - val_loss: 0.6929 - val_mean_absolute_error: 0.6929
Epoch 90/100
48/

Epoch 34/100
48/48 [==============================] - 0s 1ms/step - loss: 2.8457 - mean_absolute_error: 2.8457 - val_loss: 0.4898 - val_mean_absolute_error: 0.4898
Epoch 35/100
48/48 [==============================] - 0s 1ms/step - loss: 2.8418 - mean_absolute_error: 2.8418 - val_loss: 0.4909 - val_mean_absolute_error: 0.4909
Epoch 36/100
48/48 [==============================] - 0s 1ms/step - loss: 2.8414 - mean_absolute_error: 2.8414 - val_loss: 0.4851 - val_mean_absolute_error: 0.4851
Epoch 37/100
48/48 [==============================] - 0s 1ms/step - loss: 2.8067 - mean_absolute_error: 2.8067 - val_loss: 0.4732 - val_mean_absolute_error: 0.4732
Epoch 38/100
48/48 [==============================] - 0s 1ms/step - loss: 2.7484 - mean_absolute_error: 2.7484 - val_loss: 0.4570 - val_mean_absolute_error: 0.4570
Epoch 39/100
48/48 [==============================] - 0s 1ms/step - loss: 2.7012 - mean_absolute_error: 2.7012 - val_loss: 0.4373 - val_mean_absolute_error: 0.4373
Epoch 40/100
48/

Epoch 84/100
48/48 [==============================] - 0s 1ms/step - loss: 1.4925 - mean_absolute_error: 1.4925 - val_loss: 0.3147 - val_mean_absolute_error: 0.3147
Epoch 85/100
48/48 [==============================] - 0s 2ms/step - loss: 1.4482 - mean_absolute_error: 1.4482 - val_loss: 0.3164 - val_mean_absolute_error: 0.3164
Epoch 86/100
48/48 [==============================] - 0s 2ms/step - loss: 1.4027 - mean_absolute_error: 1.4027 - val_loss: 0.3160 - val_mean_absolute_error: 0.3160
Epoch 87/100
48/48 [==============================] - 0s 1ms/step - loss: 1.4712 - mean_absolute_error: 1.4712 - val_loss: 0.3194 - val_mean_absolute_error: 0.3194
Epoch 88/100
48/48 [==============================] - 0s 1ms/step - loss: 1.2316 - mean_absolute_error: 1.2316 - val_loss: 0.3249 - val_mean_absolute_error: 0.3249
Epoch 89/100
48/48 [==============================] - 0s 1ms/step - loss: 1.4745 - mean_absolute_error: 1.4745 - val_loss: 0.3231 - val_mean_absolute_error: 0.3231
Epoch 90/100
48/

Epoch 34/100
48/48 [==============================] - 0s 1ms/step - loss: 3.1160 - mean_absolute_error: 3.1160 - val_loss: 0.5387 - val_mean_absolute_error: 0.5387
Epoch 35/100
48/48 [==============================] - 0s 1ms/step - loss: 3.0581 - mean_absolute_error: 3.0581 - val_loss: 0.5607 - val_mean_absolute_error: 0.5607
Epoch 36/100
48/48 [==============================] - 0s 1ms/step - loss: 3.0051 - mean_absolute_error: 3.0051 - val_loss: 0.5847 - val_mean_absolute_error: 0.5847
Epoch 37/100
48/48 [==============================] - 0s 1ms/step - loss: 2.9570 - mean_absolute_error: 2.9570 - val_loss: 0.6110 - val_mean_absolute_error: 0.6110
Epoch 38/100
48/48 [==============================] - 0s 1ms/step - loss: 2.9107 - mean_absolute_error: 2.9107 - val_loss: 0.6366 - val_mean_absolute_error: 0.6366
Epoch 39/100
48/48 [==============================] - 0s 1ms/step - loss: 2.8945 - mean_absolute_error: 2.8945 - val_loss: 0.6606 - val_mean_absolute_error: 0.6606
Epoch 40/100
48/

Epoch 84/100
48/48 [==============================] - 0s 1ms/step - loss: 1.9624 - mean_absolute_error: 1.9624 - val_loss: 0.4152 - val_mean_absolute_error: 0.4152
Epoch 85/100
48/48 [==============================] - 0s 1ms/step - loss: 1.9388 - mean_absolute_error: 1.9388 - val_loss: 0.4143 - val_mean_absolute_error: 0.4143
Epoch 86/100
48/48 [==============================] - 0s 1ms/step - loss: 1.8950 - mean_absolute_error: 1.8950 - val_loss: 0.4144 - val_mean_absolute_error: 0.4144
Epoch 87/100
48/48 [==============================] - 0s 1ms/step - loss: 1.8929 - mean_absolute_error: 1.8929 - val_loss: 0.4148 - val_mean_absolute_error: 0.4148
Epoch 88/100
48/48 [==============================] - 0s 1ms/step - loss: 1.9033 - mean_absolute_error: 1.9033 - val_loss: 0.4158 - val_mean_absolute_error: 0.4158
Epoch 89/100
48/48 [==============================] - 0s 1ms/step - loss: 1.8250 - mean_absolute_error: 1.8250 - val_loss: 0.4181 - val_mean_absolute_error: 0.4181
Epoch 90/100
48/